In [ ]:
!pip install transformers

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [ ]:
!git clone https://github.com/huggingface/transformers.git

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
def read_txt(txt_path):
  with open(txt_path, 'r') as f:
    txt_string = f.readlines()
  return txt_string

In [ ]:
weights_dir = "output"

In [ ]:
#file_name = "test-data.txt"
file_name = "/content/drive/MyDrive/2022/NLP/test-data.txt"
corpus_string = read_txt(file_name)
corpus_string

[' the forethought here lie buried my thgs which if read with patice may show the strge meg of beg black here at the dag of the twtieth ctury this meg isot without terest to you gtle reader for the problem of the twtieth ctury is the problem of the color le pray you th receive my little book all charity studyg my words with me forgivg mistake d foible for sake of the faith d passi that is me d seekg the gra of truth hidd there have sought here to sketch vague certa outle the spiritual world which t thousd thousd americs live d strive first two chapters have tried to show what emcipati met to them d what was its aftermath a third chapter have poted out the slow rise of persal leadership d criticized cdidly the leader who bears the chief burd of his race to day th two other chapters have sketched swift outle the two worlds with d without the veil d thus have come to the ctral problem of trag m for life vturgow to deeper detail have two chapters studied the struggles of the massed millis 

In [ ]:
cmd = '''
python transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path gpt2 \
    --train_file {0} \
    --do_train \
    --num_train_epochs 3 \
    --overwrite_output_dir \
    --per_device_train_batch_size 2 \
    --output_dir {1}
'''.format(file_name, weights_dir)

In [ ]:
!{cmd}

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def get_model_tokenizer(weights_dir, device = 'cuda'):
    print("Loading Model ...")
    model = GPT2LMHeadModel.from_pretrained(weights_dir)
    model.to('cuda')
    print("Model Loaded ...")
    tokenizer = GPT2Tokenizer.from_pretrained(weights_dir)
    return model, tokenizer

def generate_messages(
    model,
    tokenizer,
    prompt_text,
    stop_token,
    length,
    num_return_sequences,
    temperature = 0.7,
    k=20,
    p=0.9,
    repetition_penalty = 1.0,
    device = 'cuda'
):

    MAX_LENGTH = int(10000)
    def adjust_length_to_model(length, max_sequence_length):
        if length < 0 and max_sequence_length > 0:
            length = max_sequence_length
        elif 0 < max_sequence_length < length:
            length = max_sequence_length  # No generation bigger than model size
        elif length < 0:
            length = MAX_LENGTH  # avoid infinite loop
        return length
        
    length = adjust_length_to_model(length=length, max_sequence_length=model.config.max_position_embeddings)

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")

    encoded_prompt = encoded_prompt.to(device)

    output_sequences = model.generate(
            input_ids=encoded_prompt,
            max_length=length + len(encoded_prompt[0]),
            temperature=temperature,
            top_k=k,
            top_p=p,
            repetition_penalty=repetition_penalty,
            do_sample=True,
            num_return_sequences=num_return_sequences,
        )

    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    generated_sequences = []

    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        #print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
        generated_sequence = generated_sequence.tolist()

        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # Remove all text after the stop token
        text = text[: text.find(stop_token) if stop_token else None]

        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
            prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
        )

        generated_sequences.append(total_sequence)
    return generated_sequences


In [ ]:
model, tokenizer = get_model_tokenizer(weights_dir, device = 'cuda')

Loading Model ...
Model Loaded ...


In [ ]:
model

In [ ]:
temperature = 1.0
k=400
p=0.9
repetition_penalty = 1.0
num_return_sequences = 1
length = 500
stop_token = '|EndOfText|'
prompt_text = "Jane is"

In [ ]:
%%time
generate_messages(
    model,
    tokenizer,
    prompt_text,
    stop_token,
    length,
    num_return_sequences,
    temperature = temperature,
    k=k,
    p=p,
    repetition_penalty = repetition_penalty
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 5.17 s, sys: 10.8 ms, total: 5.18 s
Wall time: 5.19 s


['Jane is a local who took care of a lot of important stuff that might have hurt her if it was really her fault. They all stuck it out to her as though it were a subject that would be self taught but she was already ready for the time of her birth which had at last happened with a special deal she had put together for us the author had prepared for us by c otricature for some twelve years later she passed over the idea with pleasure it had always pleased her to be asked at length what it was her pleasure to be asked what it was her pleasure to be asked which was more correct it was the truth she was very glad to give it a very popular revival d very grateful for my pleasure a fellow had offered it but she could hardly bear to allow herself to be compared to the idea of that imposture which we had had all raised herself to believe she would regret the time of this history almost half a century afterwards but so it was that two years afterwards it would be dear to say she had ceased to b